# catBoost
 (Categorical Boosting) is an advanced gradient boosting algorithm developed by Yandex. It is specially designed to handle categorical features efficiently without requiring heavy preprocessing like one-hot encoding. CatBoost builds an ensemble of decision trees using the gradient boosting technique and applies ordered boosting to reduce overfitting. It works well on tabular datasets containing a mix of categorical and numerical data, often outperforming other boosting algorithms like XGBoost and LightGBM in such cases.

**Key Points to Remember**

- Developed by Yandex.

- Handles categorical features directly.

- Uses ordered boosting to reduce overfitting.

- Works on classification, regression, and ranking tasks.

- Performs well on tabular data with many categorical variables.

# catBoost

In [29]:
# impoting libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report

In [30]:
# loading the data set
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# pre processing

In [31]:
df.isnull().sum()

,0
survived,0
pclass,0
sex,0
age,177
sibsp,0
parch,0
fare,0
embarked,2
class,0
who,0


# pre processing

In [32]:
# droping the deck column bcz there is high amount of null values
df.drop('deck',axis=1,inplace=True)
# handling missing value of age column using KNNimputer
from sklearn.impute import KNNImputer
imputer_age = KNNImputer(n_neighbors=10)
df['age'] = imputer_age.fit_transform(df[['age']])
# filling the missing values of embarked column by using simple imputer
from sklearn.impute import SimpleImputer
imputer_embarked = SimpleImputer(strategy='most_frequent')
df['embarked'] = imputer_embarked.fit_transform(df[['embarked']])[:,0]
df['embark_town'] = imputer_embarked.fit_transform(df[['embark_town']])[:,0]

In [33]:
df.isnull().sum()

,0
survived,0
pclass,0
sex,0
age,0
sibsp,0
parch,0
fare,0
embarked,0
class,0
who,0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     891 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  891 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB


In [35]:
# converting each category and object column into category
category_columns = df.select_dtypes(include=['category','object']).columns
# add this as a new column in the dataframe
df[category_columns] = df[category_columns].astype('category')

In [36]:
# spliting the data for X and y
X = df.drop('survived',axis=1)
y = df['survived']
# split data into train and test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [39]:
# selecting the model
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=False
)
# fiting the model
categorical_features_indices = X_train.select_dtypes(include='category').columns.tolist()
model.fit(X_train,y_train, cat_features=categorical_features_indices)
# predicting the model
y_pred = model.predict(X_test)
# evaluating the model
print(accuracy_score(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

1.0
1.0
[[105   0]
 [  0  74]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       105
           1       1.00      1.00      1.00        74

    accuracy                           1.00       179
   macro avg       1.00      1.00      1.00       179
weighted avg       1.00      1.00      1.00       179

